# **Preloaded Dataset (Use This in CSV & JSON)**

Use this dataset across all formats:


In [2]:
data = [
    (101, "Laptop", "Electronics", 55000, 10),
    (102, "Smartphone", "Electronics", 30000, 25),
    (103, "Chair", "Furniture", 2500, 50),
    (104, "Book", "Stationery", 400, 200),
    (105, "Headphones", "Electronics", 1500, 100),
    (106, "Table", "Furniture", 3200, 40),
    (107, "Pen", "Stationery", 20, 500),
    (108, "Monitor", "Electronics", 12000, 15),
    (109, "Notebook", "Stationery", 60, 300),
    (110, "Sofa", "Furniture", 45000, 5)
]

columns = ["product_id", "product_name", "category", "price", "quantity"]
df = spark.createDataFrame(data, columns)
df.write.csv("products.csv", header=True, mode="overwrite")
df.write.json("products.json", mode="overwrite")

**Tasks & Questions**

1. Read the above data from CSV into a DataFrame and print the schema.

In [3]:
df_csv = spark.read.csv("products.csv", header=True, inferSchema=True)
df_csv.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- quantity: integer (nullable = true)



2. Read the same data from JSON and compare with the CSV schema. Any differences?

In [4]:
df_json = spark.read.json("products.json")
df_json.printSchema()

root
 |-- category: string (nullable = true)
 |-- price: long (nullable = true)
 |-- product_id: long (nullable = true)
 |-- product_name: string (nullable = true)
 |-- quantity: long (nullable = true)



3. Convert the CSV data into Parquet format and save to disk.

In [5]:
df_csv.write.parquet("products.parquet", mode="overwrite")

4. Measure the size of CSV vs JSON vs Parquet on disk. Which one is smallest?

In [6]:
import os

csv_size = sum(os.path.getsize(os.path.join("products.csv", f)) for f in os.listdir("products.csv"))
json_size = sum(os.path.getsize(os.path.join("products.json", f)) for f in os.listdir("products.json"))
parquet_size = sum(os.path.getsize(os.path.join("products.parquet", f)) for f in os.listdir("products.parquet"))

print(f"CSV: {csv_size} bytes\nJSON: {json_size} bytes\nParquet: {parquet_size} bytes")

CSV: 433 bytes
JSON: 977 bytes
Parquet: 3298 bytes


5. Add a column total_revenue = price * quantity for each record.

In [7]:
from pyspark.sql.functions import col, round, _sum, avg, count, when

df = df.withColumn("total_revenue", col("price") * col("quantity"))
df.show()

+----------+------------+-----------+-----+--------+-------------+
|product_id|product_name|   category|price|quantity|total_revenue|
+----------+------------+-----------+-----+--------+-------------+
|       101|      Laptop|Electronics|55000|      10|       550000|
|       102|  Smartphone|Electronics|30000|      25|       750000|
|       103|       Chair|  Furniture| 2500|      50|       125000|
|       104|        Book| Stationery|  400|     200|        80000|
|       105|  Headphones|Electronics| 1500|     100|       150000|
|       106|       Table|  Furniture| 3200|      40|       128000|
|       107|         Pen| Stationery|   20|     500|        10000|
|       108|     Monitor|Electronics|12000|      15|       180000|
|       109|    Notebook| Stationery|   60|     300|        18000|
|       110|        Sofa|  Furniture|45000|       5|       225000|
+----------+------------+-----------+-----+--------+-------------+



6. Find the top 3 products with the highest total revenue.

In [8]:
df.orderBy(col("total_revenue").desc()).show(3)

+----------+------------+-----------+-----+--------+-------------+
|product_id|product_name|   category|price|quantity|total_revenue|
+----------+------------+-----------+-----+--------+-------------+
|       102|  Smartphone|Electronics|30000|      25|       750000|
|       101|      Laptop|Electronics|55000|      10|       550000|
|       110|        Sofa|  Furniture|45000|       5|       225000|
+----------+------------+-----------+-----+--------+-------------+
only showing top 3 rows



7. Filter and display only Furniture products with price > 3000.

In [9]:
df.filter((col("category") == "Furniture") & (col("price") > 3000)).show()

+----------+------------+---------+-----+--------+-------------+
|product_id|product_name| category|price|quantity|total_revenue|
+----------+------------+---------+-----+--------+-------------+
|       106|       Table|Furniture| 3200|      40|       128000|
|       110|        Sofa|Furniture|45000|       5|       225000|
+----------+------------+---------+-----+--------+-------------+



8. Create a new column price_band with values:
'High' if price > 10000
'Medium' if 3000 < price <= 10000
'Low' if price ≤ 3000

In [10]:
df = df.withColumn("price_band", when(col("price") > 10000, "High")
                                 .when((col("price") > 3000) & (col("price") <= 10000), "Medium")
                                 .otherwise("Low"))
df.select("product_name", "price", "price_band").show()

+------------+-----+----------+
|product_name|price|price_band|
+------------+-----+----------+
|      Laptop|55000|      High|
|  Smartphone|30000|      High|
|       Chair| 2500|       Low|
|        Book|  400|       Low|
|  Headphones| 1500|       Low|
|       Table| 3200|    Medium|
|         Pen|   20|       Low|
|     Monitor|12000|      High|
|    Notebook|   60|       Low|
|        Sofa|45000|      High|
+------------+-----+----------+



9. Group by category and calculate total quantity sold.

In [11]:
df.groupBy("category").agg(_sum("quantity").alias("total_quantity")).show()

+-----------+--------------+
|   category|total_quantity|
+-----------+--------------+
| Stationery|          1000|
|Electronics|           150|
|  Furniture|            95|
+-----------+--------------+



10. Calculate average price of products for each category.

In [12]:
df.groupBy("category").agg(round(avg("price"), 2).alias("avg_price")).show()

+-----------+---------+
|   category|avg_price|
+-----------+---------+
| Stationery|    160.0|
|Electronics|  24625.0|
|  Furniture|  16900.0|
+-----------+---------+



11. Count how many products fall in each price_band .

In [13]:
df.groupBy("price_band").agg(count("*").alias("count")).show()

+----------+-----+
|price_band|count|
+----------+-----+
|      High|    4|
|       Low|    5|
|    Medium|    1|
+----------+-----+



12. Write the filtered Electronics products (price > 5000) into a Parquet file.

In [14]:
df.filter((col("category") == "Electronics") & (col("price") > 5000)) \
  .write.parquet("electronics_filtered.parquet", mode="overwrite")

13. Write the Stationery products into a JSON file.

In [15]:
df.filter(col("category") == "Stationery") \
  .write.json("stationery_products.json", mode="overwrite")

14. Load Parquet back and run a query to find which category has highest total revenue.

In [16]:
df_parquet = spark.read.parquet("products.parquet")
df_parquet = df_parquet.withColumn("total_revenue", col("price") * col("quantity"))
df_parquet.groupBy("category").agg(_sum("total_revenue").alias("category_revenue")) \
    .orderBy(col("category_revenue").desc()).show(1)

+-----------+----------------+
|   category|category_revenue|
+-----------+----------------+
|Electronics|         1630000|
+-----------+----------------+
only showing top 1 row



15. BONUS: Create a temporary view from the DataFrame and run Spark SQL to find all
products with quantity > 100 and price < 1000.

In [17]:
df.createOrReplaceTempView("products_view")
spark.sql("SELECT * FROM products_view WHERE quantity > 100 AND price < 1000").show()

+----------+------------+----------+-----+--------+-------------+----------+
|product_id|product_name|  category|price|quantity|total_revenue|price_band|
+----------+------------+----------+-----+--------+-------------+----------+
|       104|        Book|Stationery|  400|     200|        80000|       Low|
|       107|         Pen|Stationery|   20|     500|        10000|       Low|
|       109|    Notebook|Stationery|   60|     300|        18000|       Low|
+----------+------------+----------+-----+--------+-------------+----------+

